# SparkSession
Objetivo: crear una Spark session, entrada a las funcionalidades de spark. 
Con base en el video y en el repositorio de Errodringer, pero con la adecuación y adaptación a mi version de pyspark y python. 

https://github.com/errodringer/CursoBigData_PySpark/tree/master
https://www.youtube.com/playlist?list=PLYFBiuYObvKB_k0bnkI41biMjD12t44GN

Nota: para iniciar el jupyter notebook como en el vídeo, yo en mi consola puse el siguiente comando porque no me lo reconoce. 

python -m notebook

In [1]:
import findspark 
findspark.init()
import pyspark.sql.functions as F
import pandas as pd
import numpy as np  

from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, split, when, abs, row_number, monotonically_increasing_id
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
spark = SparkSession.builder.appName ('firstSession').getOrCreate()

Todas las configuraciones disponibles en Spark:
https://spark.apache.org/docs/latest/configuration.html

In [3]:
spark

# DataFrame

En Python, un DataFrame es una estructura de datos bidimensional que se encuentra en la biblioteca pandas. Está diseñado para almacenar y manipular datos de manera similar a una hoja de cálculo o una tabla de base de datos relacional. 
No me dejo crear el df a partir de una lista si no importaba pandas, busque los mil errores y esta fue una solición. 

Crear Tabla
a) A partir de una lista 

En este caso se utilizo pandas aunque no es lo más recomendable porque se utilizan más recursos de la computadora, por lo que tendríamos procesamientos más lentos. 


In [4]:
columnas = ["id", "nombre", "letra"]
lista_1 = [(1, "Errodringer", "a"), (2, "Paco", "b"), (3, "Hola", "c"), (4, "Adios", "d")]

In [5]:
df_1 = pd.DataFrame(lista_1, columns= columnas)
df_1

,id,nombre,letra
0,1,Errodringer,a
1,2,Paco,b
2,3,Hola,c
3,4,Adios,d


b) A partir de un CSV o txt 

Es uno de los formatos más comunes, de manera general es df = spark.read.formato(path) aunque no son los únicos formatos ni las unicas variantes. Con el .csv me toma las primeras dos lineas como el header y en el .txt no reconoce el header. 

Para este ejemplo al leer el csv y mostrarlo con show, se observa que coloca los datos en una sóla columna, por lo que se utiliza la funcion .withColumn para reesctibir el schema. Se probaron otras formas de inferir el schema y hacer directamente un split pero ninguna funciono a que el resultado fuera, dos columnas con encabezado.Los numeros de la  función withColumn indican las posiciones que toma la columna que se crea y toma en cuenta los espacios. 


En el caso que se encuentren separados por comas tendría que buscar el comando que reconozca esta separación, por el momento me manejo sólamente por posiciones y espacios de columna. 

Coloque el .drop directamente al nombrar las columnas aunque se puede poner fuera. Simplemente corta y elimina la columna de los datos crudos. 


In [6]:
df_2 = spark.read.csv("data\\DBS_LW_ModA_highres.txt", header=True, inferSchema=True )
df_2.printSchema()


root
 |-- microns  transmission: string (nullable = true)



In [7]:
df_2 = df_2.withColumn("microns", F.substring(F.col("microns  transmission"),0,7))\
    .withColumn("transmission", F.substring(F.col("microns  transmission"),7,15))\
    .drop("microns  transmission")
df_2.printSchema()
df_2.count()
df_2.first()
df_2.show(50)
df_2.collect()
nombres_columnas = df_2.columns
num_columns = len(df_2.columns)
print("Número de columnas:", num_columns)


root
 |-- microns: string (nullable = true)
 |-- transmission: string (nullable = true)

+-------+------------+
|microns|transmission|
+-------+------------+
|2      | 1.93996E-05|
|2.001  | 1.79938E-05|
|2.002  | 1.67365E-05|
|2.003  | 1.56101E-05|
|2.004  | 1.45996E-05|
|2.005  | 1.36915E-05|
|2.006  | 1.28742E-05|
|2.007  | 1.21374E-05|
|2.008  | 1.14722E-05|
|2.009  | 1.08703E-05|
|2.01   | 1.03247E-05|
|2.011  | 9.82864E-06|
|2.012  | 9.37634E-06|
|2.013  | 8.96236E-06|
|2.014  | 8.58179E-06|
|2.015  | 8.23018E-06|
|2.016  | 7.90358E-06|
|2.017  | 7.59852E-06|
|2.018  | 7.31207E-06|
|2.019  | 7.04185E-06|
|2.02   | 6.78604E-06|
|2.021  | 6.54336E-06|
|2.022  | 6.31297E-06|
|2.023  | 6.09444E-06|
|2.024  | 5.88758E-06|
|2.025  | 5.69239E-06|
|2.026  | 5.50893E-06|
|2.027  | 5.33726E-06|
|2.028  | 5.17737E-06|
|2.029  | 5.02919E-06|
|2.03   | 4.89252E-06|
|2.031  | 4.76713E-06|
|2.032  | 4.65268E-06|
|2.033  | 4.54879E-06|
|2.034  | 4.45506E-06|
|2.035  | 4.37107E-06|
|2.036  |  4.2

# Filtros

In [8]:
df_2.filter((col('microns') > 2) & (col('transmission') .between(0.935, 0.937))
).show()

+-------+------------+
|microns|transmission|
+-------+------------+
|3.004  | 0.935388099|
|3.005  | 0.935807141|
|3.006  | 0.936212417|
|3.007  | 0.936603099|
|3.008  | 0.936978404|
|3.05   | 0.936888018|
|3.051  | 0.936645444|
|3.052  | 0.936403758|
|3.053  | 0.936163889|
|3.054  | 0.935926744|
|3.055  | 0.935693212|
|3.056  | 0.935464161|
|3.057  | 0.935240431|
|3.058  | 0.935022836|
|3.094  | 0.935184363|
|3.095  | 0.935431772|
|3.096  | 0.935689107|
|3.097  | 0.935955849|
|3.098  | 0.936231463|
|3.099  | 0.936515399|
+-------+------------+
only showing top 20 rows



Ordenar en descendente y ascendente 

.orderBy también permite ordenar alfabéticamente, es más recomendable utilizar orderBy por cuestiones de sintaxis y legibilidad, compatibilidad, rendimiento. 

In [9]:
df_2_descendente =df_2.orderBy(df_2["microns"].desc()).show(50)

+-------+------------+
|microns|transmission|
+-------+------------+
|5.25   | 0.830337544|
|5.249  | 0.830896957|
|5.248  | 0.831456975|
|5.247  | 0.832017537|
|5.246  |  0.83257858|
|5.245  | 0.833140039|
|5.244  | 0.833701853|
|5.243  | 0.834263956|
|5.242  | 0.834826287|
|5.241  |  0.83538878|
|5.24   | 0.835951372|
|5.239  | 0.836513999|
|5.238  | 0.837076597|
|5.237  | 0.837639103|
|5.236  | 0.838201451|
|5.235  | 0.838763579|
|5.234  | 0.839325421|
|5.233  | 0.839886915|
|5.232  | 0.840447997|
|5.231  | 0.841008602|
|5.23   | 0.841568667|
|5.229  | 0.842128129|
|5.228  | 0.842686925|
|5.227  | 0.843244991|
|5.226  | 0.843802264|
|5.225  | 0.844358682|
|5.224  | 0.844914183|
|5.223  | 0.845468703|
|5.222  | 0.846022182|
|5.221  | 0.846574558|
|5.22   |  0.84712577|
|5.219  | 0.847675757|
|5.218  | 0.848224458|
|5.217  | 0.848771813|
|5.216  | 0.849317764|
|5.215  | 0.849862249|
|5.214  | 0.850405212|
|5.213  | 0.850946593|
|5.212  | 0.851486335|
|5.211  | 0.852024381|
|5.21   | 0

In [10]:
df_2_ascendente =df_2.orderBy(df_2["microns"]).show(50)

+-------+------------+
|microns|transmission|
+-------+------------+
|2      | 1.93996E-05|
|2.001  | 1.79938E-05|
|2.002  | 1.67365E-05|
|2.003  | 1.56101E-05|
|2.004  | 1.45996E-05|
|2.005  | 1.36915E-05|
|2.006  | 1.28742E-05|
|2.007  | 1.21374E-05|
|2.008  | 1.14722E-05|
|2.009  | 1.08703E-05|
|2.01   | 1.03247E-05|
|2.011  | 9.82864E-06|
|2.012  | 9.37634E-06|
|2.013  | 8.96236E-06|
|2.014  | 8.58179E-06|
|2.015  | 8.23018E-06|
|2.016  | 7.90358E-06|
|2.017  | 7.59852E-06|
|2.018  | 7.31207E-06|
|2.019  | 7.04185E-06|
|2.02   | 6.78604E-06|
|2.021  | 6.54336E-06|
|2.022  | 6.31297E-06|
|2.023  | 6.09444E-06|
|2.024  | 5.88758E-06|
|2.025  | 5.69239E-06|
|2.026  | 5.50893E-06|
|2.027  | 5.33726E-06|
|2.028  | 5.17737E-06|
|2.029  | 5.02919E-06|
|2.03   | 4.89252E-06|
|2.031  | 4.76713E-06|
|2.032  | 4.65268E-06|
|2.033  | 4.54879E-06|
|2.034  | 4.45506E-06|
|2.035  | 4.37107E-06|
|2.036  |  4.2964E-06|
|2.037  | 4.23064E-06|
|2.038  | 4.17341E-06|
|2.039  | 4.12436E-06|
|2.04   | 4

In [11]:
df_2_descendente_sort = df_2.sort(df_2["microns"].desc()).show(50)

+-------+------------+
|microns|transmission|
+-------+------------+
|5.25   | 0.830337544|
|5.249  | 0.830896957|
|5.248  | 0.831456975|
|5.247  | 0.832017537|
|5.246  |  0.83257858|
|5.245  | 0.833140039|
|5.244  | 0.833701853|
|5.243  | 0.834263956|
|5.242  | 0.834826287|
|5.241  |  0.83538878|
|5.24   | 0.835951372|
|5.239  | 0.836513999|
|5.238  | 0.837076597|
|5.237  | 0.837639103|
|5.236  | 0.838201451|
|5.235  | 0.838763579|
|5.234  | 0.839325421|
|5.233  | 0.839886915|
|5.232  | 0.840447997|
|5.231  | 0.841008602|
|5.23   | 0.841568667|
|5.229  | 0.842128129|
|5.228  | 0.842686925|
|5.227  | 0.843244991|
|5.226  | 0.843802264|
|5.225  | 0.844358682|
|5.224  | 0.844914183|
|5.223  | 0.845468703|
|5.222  | 0.846022182|
|5.221  | 0.846574558|
|5.22   |  0.84712577|
|5.219  | 0.847675757|
|5.218  | 0.848224458|
|5.217  | 0.848771813|
|5.216  | 0.849317764|
|5.215  | 0.849862249|
|5.214  | 0.850405212|
|5.213  | 0.850946593|
|5.212  | 0.851486335|
|5.211  | 0.852024381|
|5.21   | 0

In [12]:
df_2_index = df_2.sort(df_2["microns"].desc()).show(50)

+-------+------------+
|microns|transmission|
+-------+------------+
|5.25   | 0.830337544|
|5.249  | 0.830896957|
|5.248  | 0.831456975|
|5.247  | 0.832017537|
|5.246  |  0.83257858|
|5.245  | 0.833140039|
|5.244  | 0.833701853|
|5.243  | 0.834263956|
|5.242  | 0.834826287|
|5.241  |  0.83538878|
|5.24   | 0.835951372|
|5.239  | 0.836513999|
|5.238  | 0.837076597|
|5.237  | 0.837639103|
|5.236  | 0.838201451|
|5.235  | 0.838763579|
|5.234  | 0.839325421|
|5.233  | 0.839886915|
|5.232  | 0.840447997|
|5.231  | 0.841008602|
|5.23   | 0.841568667|
|5.229  | 0.842128129|
|5.228  | 0.842686925|
|5.227  | 0.843244991|
|5.226  | 0.843802264|
|5.225  | 0.844358682|
|5.224  | 0.844914183|
|5.223  | 0.845468703|
|5.222  | 0.846022182|
|5.221  | 0.846574558|
|5.22   |  0.84712577|
|5.219  | 0.847675757|
|5.218  | 0.848224458|
|5.217  | 0.848771813|
|5.216  | 0.849317764|
|5.215  | 0.849862249|
|5.214  | 0.850405212|
|5.213  | 0.850946593|
|5.212  | 0.851486335|
|5.211  | 0.852024381|
|5.21   | 0

Reemplazar elementos. 

In [13]:
df_2con_cambio = df_2.withColumn("microns",
                               when(df_2["microns"] == 2, 3)
                               .otherwise(df_2["microns"])).show(50)

+-------+------------+
|microns|transmission|
+-------+------------+
|      3| 1.93996E-05|
|      3| 1.79938E-05|
|      3| 1.67365E-05|
|      3| 1.56101E-05|
|      3| 1.45996E-05|
|      3| 1.36915E-05|
|      3| 1.28742E-05|
|      3| 1.21374E-05|
|      3| 1.14722E-05|
|      3| 1.08703E-05|
|      3| 1.03247E-05|
|      3| 9.82864E-06|
|      3| 9.37634E-06|
|      3| 8.96236E-06|
|      3| 8.58179E-06|
|      3| 8.23018E-06|
|      3| 7.90358E-06|
|      3| 7.59852E-06|
|      3| 7.31207E-06|
|      3| 7.04185E-06|
|      3| 6.78604E-06|
|      3| 6.54336E-06|
|      3| 6.31297E-06|
|      3| 6.09444E-06|
|      3| 5.88758E-06|
|      3| 5.69239E-06|
|      3| 5.50893E-06|
|      3| 5.33726E-06|
|      3| 5.17737E-06|
|      3| 5.02919E-06|
|      3| 4.89252E-06|
|      3| 4.76713E-06|
|      3| 4.65268E-06|
|      3| 4.54879E-06|
|      3| 4.45506E-06|
|      3| 4.37107E-06|
|      3|  4.2964E-06|
|      3| 4.23064E-06|
|      3| 4.17341E-06|
|      3| 4.12436E-06|
|      3| 4

Nota del Chat GPT: 

1.-df_2["microns"]: Esto accede a la columna 'microns' del DataFrame df_2.

2.-abs(df_2["microns"] - 2.0): Aquí, estamos calculando la diferencia absoluta entre cada valor en la columna 'microns' y el valor 2.0. Esto nos da la distancia absoluta entre cada valor y 2.0.

3.- < tolerancia: Luego, comparamos cada una de estas diferencias absolutas con una tolerancia predefinida (tolerancia). Si la diferencia absoluta es menor que la tolerancia, significa que el valor es cercano a 2.0.



In [14]:
tolerancia = 0.0001
df_2_tolerancia = df_2.withColumn("microns",
                               when(abs(df_2["microns"] - 2.0) < tolerancia, 3)
                               .otherwise(df_2["microns"])).show(50)

+-------+------------+
|microns|transmission|
+-------+------------+
|      3| 1.93996E-05|
|2.001  | 1.79938E-05|
|2.002  | 1.67365E-05|
|2.003  | 1.56101E-05|
|2.004  | 1.45996E-05|
|2.005  | 1.36915E-05|
|2.006  | 1.28742E-05|
|2.007  | 1.21374E-05|
|2.008  | 1.14722E-05|
|2.009  | 1.08703E-05|
|2.01   | 1.03247E-05|
|2.011  | 9.82864E-06|
|2.012  | 9.37634E-06|
|2.013  | 8.96236E-06|
|2.014  | 8.58179E-06|
|2.015  | 8.23018E-06|
|2.016  | 7.90358E-06|
|2.017  | 7.59852E-06|
|2.018  | 7.31207E-06|
|2.019  | 7.04185E-06|
|2.02   | 6.78604E-06|
|2.021  | 6.54336E-06|
|2.022  | 6.31297E-06|
|2.023  | 6.09444E-06|
|2.024  | 5.88758E-06|
|2.025  | 5.69239E-06|
|2.026  | 5.50893E-06|
|2.027  | 5.33726E-06|
|2.028  | 5.17737E-06|
|2.029  | 5.02919E-06|
|2.03   | 4.89252E-06|
|2.031  | 4.76713E-06|
|2.032  | 4.65268E-06|
|2.033  | 4.54879E-06|
|2.034  | 4.45506E-06|
|2.035  | 4.37107E-06|
|2.036  |  4.2964E-06|
|2.037  | 4.23064E-06|
|2.038  | 4.17341E-06|
|2.039  | 4.12436E-06|
|2.04   | 4

In [15]:
nuevo_valor = 100
valor_t = 200
df_2_cambio_fila = df_2.selectExpr(
    "CASE WHEN ROW_NUMBER() OVER (ORDER BY NULL) = 3 THEN {} ELSE microns END AS microns".format(nuevo_valor),
    "CASE WHEN ROW_NUMBER() OVER (ORDER BY NULL) = 3 THEN {} ELSE transmission END AS transmission".format(valor_t)
).show(50)

+-------+------------+
|microns|transmission|
+-------+------------+
|2      | 1.93996E-05|
|2.001  | 1.79938E-05|
|    100|         200|
|2.003  | 1.56101E-05|
|2.004  | 1.45996E-05|
|2.005  | 1.36915E-05|
|2.006  | 1.28742E-05|
|2.007  | 1.21374E-05|
|2.008  | 1.14722E-05|
|2.009  | 1.08703E-05|
|2.01   | 1.03247E-05|
|2.011  | 9.82864E-06|
|2.012  | 9.37634E-06|
|2.013  | 8.96236E-06|
|2.014  | 8.58179E-06|
|2.015  | 8.23018E-06|
|2.016  | 7.90358E-06|
|2.017  | 7.59852E-06|
|2.018  | 7.31207E-06|
|2.019  | 7.04185E-06|
|2.02   | 6.78604E-06|
|2.021  | 6.54336E-06|
|2.022  | 6.31297E-06|
|2.023  | 6.09444E-06|
|2.024  | 5.88758E-06|
|2.025  | 5.69239E-06|
|2.026  | 5.50893E-06|
|2.027  | 5.33726E-06|
|2.028  | 5.17737E-06|
|2.029  | 5.02919E-06|
|2.03   | 4.89252E-06|
|2.031  | 4.76713E-06|
|2.032  | 4.65268E-06|
|2.033  | 4.54879E-06|
|2.034  | 4.45506E-06|
|2.035  | 4.37107E-06|
|2.036  |  4.2964E-06|
|2.037  | 4.23064E-06|
|2.038  | 4.17341E-06|
|2.039  | 4.12436E-06|
|2.04   | 4

In [16]:
df_2_r = df_2.selectExpr(
    "CASE WHEN ROW_NUMBER() OVER (ORDER BY NULL) = 3 THEN {} ELSE microns END AS microns".format(nuevo_valor),
    "CASE WHEN ROW_NUMBER() OVER (ORDER BY NULL) = 3 THEN {} ELSE transmission END AS transmission".format(valor_t)
).show(50,  truncate=False)

+-------+------------+
|microns|transmission|
+-------+------------+
|2      | 1.93996E-05|
|2.001  | 1.79938E-05|
|100    |200         |
|2.003  | 1.56101E-05|
|2.004  | 1.45996E-05|
|2.005  | 1.36915E-05|
|2.006  | 1.28742E-05|
|2.007  | 1.21374E-05|
|2.008  | 1.14722E-05|
|2.009  | 1.08703E-05|
|2.01   | 1.03247E-05|
|2.011  | 9.82864E-06|
|2.012  | 9.37634E-06|
|2.013  | 8.96236E-06|
|2.014  | 8.58179E-06|
|2.015  | 8.23018E-06|
|2.016  | 7.90358E-06|
|2.017  | 7.59852E-06|
|2.018  | 7.31207E-06|
|2.019  | 7.04185E-06|
|2.02   | 6.78604E-06|
|2.021  | 6.54336E-06|
|2.022  | 6.31297E-06|
|2.023  | 6.09444E-06|
|2.024  | 5.88758E-06|
|2.025  | 5.69239E-06|
|2.026  | 5.50893E-06|
|2.027  | 5.33726E-06|
|2.028  | 5.17737E-06|
|2.029  | 5.02919E-06|
|2.03   | 4.89252E-06|
|2.031  | 4.76713E-06|
|2.032  | 4.65268E-06|
|2.033  | 4.54879E-06|
|2.034  | 4.45506E-06|
|2.035  | 4.37107E-06|
|2.036  | 4.2964E-06 |
|2.037  | 4.23064E-06|
|2.038  | 4.17341E-06|
|2.039  | 4.12436E-06|
|2.04   | 4

Nota Chat GPT: La función .cast() en PySpark se utiliza para convertir el tipo de datos de una columna a otro tipo de datos.

Por ejemplo, si tienes una columna que contiene valores en formato de cadena ("string") y deseas tratar esos valores como números enteros, puedes usar .cast("int") para convertir la columna a tipo entero. Del mismo modo, si deseas tratar los valores como números de punto flotante, puedes usar .cast("float").

In [17]:
# Definir el rango de filas que quieres reemplazar
inicio_fila = 3
fin_fila = 15

# Nuevo valor que deseas asignar a las filas dentro del rango
nuevo_valor = 100

# Crear una ventana para numerar las filas
windowSpec = Window.orderBy(F.monotonically_increasing_id())
type(windowSpec)
print(windowSpec)
# Seleccionar las filas y reemplazar los valores en el rango especificado
df_2_rem = df_2.withColumn("microns",
                            when((row_number().over(windowSpec).between(inicio_fila, fin_fila)), nuevo_valor)
                            .otherwise(F.col("microns")))

df_2_rem =df_2_rem.withColumn("transmission",
                            when((row_number().over(windowSpec).between(inicio_fila, fin_fila)), valor_t)
                            .otherwise(F.col("transmission")))

# Mostrar el DataFrame resultante
df_2_rem.show(truncate=False)

+-------+------------+
|microns|transmission|
+-------+------------+
|2      | 1.93996E-05|
|2.001  | 1.79938E-05|
|100    |200         |
|100    |200         |
|100    |200         |
|100    |200         |
|100    |200         |
|100    |200         |
|100    |200         |
|100    |200         |
|100    |200         |
|100    |200         |
|100    |200         |
|100    |200         |
|100    |200         |
|2.015  | 8.23018E-06|
|2.016  | 7.90358E-06|
|2.017  | 7.59852E-06|
|2.018  | 7.31207E-06|
|2.019  | 7.04185E-06|
+-------+------------+
only showing top 20 rows



when: Función que permite aplicar una condición a una expresión y devolver un valor específico si la condición es verdadera, y otro valor si la condición es falsa.

row_number: Función de ventana que asigna un número de fila a cada fila del DataFrame.

Window: Clase que define una ventana de filas basada en algún criterio de ordenamiento.

Window.orderBy(F.monotonically_increasing_id()) crea una ventana que ordena las filas según un identificador único de fila.

when((row_number().over(windowSpec).between(inicio_fila, fin_fila)), nuevo_valor): Esta es una condición que se evalúa para cada fila del DataFrame. Utiliza la función row_number() para numerar las filas, y luego verifica si el número de fila está dentro del rango especificado por inicio_fila y fin_fila. Si la fila está dentro de ese rango, se asigna el valor nuevo_valor a la columna "microns".

.otherwise(F.col("microns")): Si la condición de when() no se cumple para una fila en particular (es decir, si la fila está fuera del rango especificado), se mantiene el valor original de la columna "microns" para esa fila

In [18]:
df_2_rem.show()


+-------+------------+
|microns|transmission|
+-------+------------+
|2      | 1.93996E-05|
|2.001  | 1.79938E-05|
|    100|         200|
|    100|         200|
|    100|         200|
|    100|         200|
|    100|         200|
|    100|         200|
|    100|         200|
|    100|         200|
|    100|         200|
|    100|         200|
|    100|         200|
|    100|         200|
|    100|         200|
|2.015  | 8.23018E-06|
|2.016  | 7.90358E-06|
|2.017  | 7.59852E-06|
|2.018  | 7.31207E-06|
|2.019  | 7.04185E-06|
+-------+------------+
only showing top 20 rows



In [19]:

rdd = spark.sparkContext.parallelize(df_2_rem.select("*").collect()[1:15]).collect()
print (rdd)
type(rdd)

[Row(microns='2.001  ', transmission=' 1.79938E-05'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200'), Row(microns='100', transmission='200')]


list

In [60]:
df_result = df_2_rem.withColumn("row_id", monotonically_increasing_id())
df_result = df_result.filter((df_result["row_id"] > 3) & (df_result["row_id"] < 15))
df_result.show()




+-------+------------+------+
|microns|transmission|row_id|
+-------+------------+------+
|    100|         200|     4|
|    100|         200|     5|
|    100|         200|     6|
|    100|         200|     7|
|    100|         200|     8|
|    100|         200|     9|
|    100|         200|    10|
|    100|         200|    11|
|    100|         200|    12|
|    100|         200|    13|
|    100|         200|    14|
+-------+------------+------+



tercera prueba